In [69]:
# encoding:utf-8
import numpy as np
import pandas as pd
import time

df1 = pd.DataFrame([['m1703-1000',1,2,'m1703'],['m1703-2000',3,4,'m1703'], \
                    ['m1705-1000',5,6,'m1705'], ['m1705-2000',7,8,'m1705']],columns=['oSymbol','x1','x2','fSymbol'], \
                    index=['m1703-C-1000','m1703-C-2000','m1705-C-1000','m1705-C-2000'])

df2 = pd.DataFrame([['m1703-1000',11,12,'m1703'],['m1703-2000',13,14,'m1703'], \
                    ['m1705-1000',15,16,'m1705'], ['m1705-2000',17,18,'m1705']],columns=['oSymbol','x1','x2','fSymbol'], \
                    index=['m1703-P-1000','m1703-P-2000','m1705-P-1000','m1705-P-2000'])

df3 = pd.DataFrame([['m1701',21,22],['m1703',23,24],['m1705',25,26]],columns=['fSymbol','x1','x2'], \
                    index=['m1701','m1703','m1705'])

print(df1)
print(df2)
tdf = pd.merge(df1, df2, on='oSymbol', how='inner', right_index=True, left_index=False, suffixes=['_x','_y'])
print(tdf)
print(df3)
tdf = pd.merge(tdf, df3, left_on='fSymbol_x', right_on='fSymbol', how='inner', right_index=True, suffixes=['', '_f'])
print(tdf)

                 oSymbol  x1  x2 fSymbol
m1703-C-1000  m1703-1000   1   2   m1703
m1703-C-2000  m1703-2000   3   4   m1703
m1705-C-1000  m1705-1000   5   6   m1705
m1705-C-2000  m1705-2000   7   8   m1705
                 oSymbol  x1  x2 fSymbol
m1703-P-1000  m1703-1000  11  12   m1703
m1703-P-2000  m1703-2000  13  14   m1703
m1705-P-1000  m1705-1000  15  16   m1705
m1705-P-2000  m1705-2000  17  18   m1705
                 oSymbol  x1_x  x2_x fSymbol_x  x1_y  x2_y fSymbol_y
m1703-C-1000  m1703-1000     1     2     m1703    11    12     m1703
m1703-C-2000  m1703-2000     3     4     m1703    13    14     m1703
m1705-C-1000  m1705-1000     5     6     m1705    15    16     m1705
m1705-C-2000  m1705-2000     7     8     m1705    17    18     m1705
      fSymbol  x1  x2
m1701   m1701  21  22
m1703   m1703  23  24
m1705   m1705  25  26
                 oSymbol  x1_x  x2_x fSymbol_x  x1_y  x2_y fSymbol_y fSymbol  \
m1703-C-1000  m1703-1000     1     2     m1703    11    12     m1703   m1703 

In [3]:
# encoding:utf-8

import shelve
import numpy as np
import pandas as pd
import time

f = shelve.open('ticksFrame.vt')
if 'data' in f:
	ticksFrame = f['data']
f.close()
ticksFrame.to_csv('ticksFrame.csv')

start = time.clock()
ticksFrame.loc[:, 'spread'] = abs(ticksFrame.loc[:, 'askPrice1'] - ticksFrame.loc[:, 'bidPrice1'])
cTicksFrame = ticksFrame[ticksFrame.vtType == 'C']
pTicksFrame = ticksFrame[ticksFrame.vtType == 'P']
fTicksFrame = ticksFrame[ticksFrame.vtType == 'F']


tdf = pd.merge(cTicksFrame, pTicksFrame, left_index=False, right_index=True, on='oSymbol', how='outer', suffixes=('_C','_P'), sort=False)
tdf.to_csv('merge_two.csv')
tdf = pd.merge(tdf, fTicksFrame, left_index=False, right_index=True, left_on='futureSymbol_C', right_on='futureSymbol', how='outer', suffixes=('', '_F'), sort=True)
tdf.to_csv('merge_three.csv')

#计算套利空间：按照市价买入、减去预计spread、MI、减去交易手续费、减去追单可能以及大小
#正向套利
tdf.loc[:, 'carraySpace'] = tdf.loc[:, 'bidPrice1_C'] - tdf.loc[:, 'askPrice1_P'] \
- tdf.loc[:, 'askPrice1'] + tdf.loc[:, 'benchMark_C'] - 7.0 \
- tdf.loc[:, 'spread_C'] - tdf.loc[:, 'spread_C'] - tdf.loc[:, 'spread_C']
#反向套利
tdf.loc[:, 'reverseSpace'] = tdf.loc[:, 'bidPrice1'] - tdf.loc[:, 'strikePrice_C'] \
- tdf.loc[:, 'askPrice1_C'] + tdf.loc[:, 'bidPrice1_P'] - 7.0 \
- tdf.loc[:, 'spread_C']- tdf.loc[:, 'spread_C']- tdf.loc[:, 'spread_C']
end = time.clock()
print u"所有使用时间: %f s" % (end - start)
tdf.to_csv('merge_finish.csv')

#得到套利空间结果
allResult = tdf[(tdf.spread_C<=2) & (tdf.spread_P<=2) & (tdf.spread<=2) & \
            (tdf.openInterest_C>=5000) & (tdf.openInterest_P>=5000) & (tdf.openInterest>=5000)].copy()
allResult.to_csv('allResult.csv')
cResult = allResult[allResult.carraySpace>=20].copy()
pResult = allResult[allResult.reverseSpace>=10].copy()
print(cResult.head())
print(pResult.head())
cResult.loc[:, 'minVolume'] = 3
cResult.loc[:, 'minVolume'] = cResult.loc[:, ['minVolume', 'bidVolume1_C', 'askVolume1_P', 'askVolume1']].min(axis=1)

pResult.loc[:, 'minVolume'] = 3
pResult.loc[:, 'minVolume'] = pResult.loc[:, ['bidVolume1', 'bidVolume1_P', 'askVolume1_C']].min(axis=1)


所有使用时间: 0.029860 s
             askPrice1_C bidPrice1_C askVolume1_C bidVolume1_C spread_C  \
m1705-C-2450         775         774          578          190        1   
m1705-C-2600         686         684            4           14        2   
m1705-C-2650       668.5       666.5            1           15        2   
m1705-C-2700       635.5         634            1            7      1.5   
m1705-C-2800         599         597            2           14        2   

             lastPrice_C volume_C openInterest_C futureSymbol_C vtType_C  \
m1705-C-2450         774    68954         243228          m1705        C   
m1705-C-2600         684     1724          29866          m1705        C   
m1705-C-2650         667      880          47510          m1705        C   
m1705-C-2700       635.5     1190          38220          m1705        C   
m1705-C-2800         597     1168          47398          m1705        C   

                 ...      openInterest futureSymbol vtType strikePrice  \

ValueError: cannot set a frame with no defined index and a scalar

In [23]:
# encoding:utf-8

import shelve
import numpy as np
import pandas as pd
import time

df = pd.DataFrame([], columns=['x1', 'x2'])

df.loc[:, 'x2'] =3
print(df)

Empty DataFrame
Columns: [x1, x2]
Index: []
